In [1]:
from meetup import meetups_api as meetup
from meetup import tidy_meetup as get_df
import numpy as np
import pandas as pd
import urllib
import json
import time
import requests
#mybinder

In [10]:
key = "2e45d6103a21861521237661ae27" #get key from https://www.meetup.com/meetup_api/ you can just open an account
# then look for the tab API key then you ready to go
places = np.load('main_cities.npy')

### Import the groups data csv 

In [12]:
data = pd.read_csv('Loneliness group key words.csv')
data.head()

,Children (< 16 years),Young adults (16 - 24 years),Adults (25 - 49 years),Older people (> 50 years),Welsh translation,Polish translation
0,Sport,Sport,Sport,Sport,Chwaraeon,Sport
1,NaN,Yoga,Yoga,Yoga,Yoga,Joga
2,NaN,Bokwa,Bokwa,Bokwa,Bokwa,Bokwa
3,NaN,Fitness,Fitness,Fitness,Ffitrwydd,Zdatność
4,Gym,Gym,Gym,Gym,Campfa,Siłownia


### Concatenating all the different colmuns:
* this piece of code drops all the duplicates, drops all NA values and sorts them in alphabetical order

In [13]:
# all Groups in the English Language
data_1 = pd.concat([data['Children (< 16 years)'],
                   data['Young adults (16 - 24 years)'],
                   data['Adults (25 - 49 years)'],
                   data['Older people (> 50 years)']
                  ]).drop_duplicates().dropna().sort_values().reset_index(drop=True)

In [14]:
# all Groups in Welsh and Polish
data_2 = pd.concat([data['Welsh translation'],
                   data['Polish translation']
                  ]).drop_duplicates().dropna().sort_values().reset_index(drop=True)

In [17]:
len(data_1)

90

In [15]:
#just to see what it looks like
#this function will also save the raw data in a txt into your current working dir
meetup(places[5:10], 'marathon', key, search_radius=100)

----5 urls created----


KeyboardInterrupt: 

### Getting everything into a nice DataFrame
* call function **get_df()** which put your data into a DataFrame

In [3]:
get_df('mum')

--(meetup_mum.npy) doesn't exist--
--please check/move file into current working directory--


### Loop through all the topics

In [10]:
import urllib3
import os
import time
import facebook
from facebook import GraphAPIError
import requests
import json
import numpy as np
import pandas as pd

token = 'EAADe3adNZCDABAA5yvwpiND92QZBV9Y7FqUliL2WfCS0cgtkTb7CamZCNfqssC6rijoo1FZBMzMSd8zgxDkx9aidWJAYShQiOjDCo1E4h3iSOCvZAxXkQSBia9ZCuvB5i8IAAwXQtepD06pZAvOPx0RFJFTTCx8kbt2x8oP8I7wkZCz7dXGrZAlXI'


def fb_pagefinder(activity, center, search_radius, token=token):
    lat = center[0]
    lng = center[1]
#     try:
    graph = facebook.GraphAPI(access_token=token, timeout=200)
    events = graph.request(
        'search?q=events%20{}&fields=name,id,description,location&type=place&center={},{}&distance={}&limit=100'.format(
        activity, lat, lng, search_radius))
#     except GraphAPIError:
#         raise
#         time.sleep(5)
#         graph = facebook.GraphAPI(access_token=token, timeout=200)
#         events = graph.request(
#     'search?q=events%20{}&fields=name,id,description,location&type=place&center={},{}&distance={}&limit=100'.format(
#         activity, lat, lng, search_radius))
    return events


def fb_api(activity, center, search_radius):
    events = []
    if cenr != 'Nan':
        search = fb_pagefinder(activity, center, search_radius)
        time.sleep(5)
        if len(events['data']) == 0:
            pass
        else:
            events.append(search)
            geolocator = Nominatim()
            location = geolocator.geocode(
                [center[0], center[1]], timeout=120)
            print(counter, "--" + location.address[:70])
    else:
        pass
    return events


def pagination(o, activity):
    all_events = []
    if ('paging' in o):
#         for data in o:
        if ('next' in o['paging']):
            resp = requests.get(o['paging']['next'])
            all_events.append(resp.json())
        elif ('after' in o['paging']['cursors']):
            new_url = URL + o['paging']['cursors']['after']
            resp = requests.get(new_url)
            all_events.append(resp.json())
        else:
             print("Something went wrong")

    new_npy = np.load('facebook_data/fb_%s.npy'%activity) if os.path.isfile('facebook_data/fb_%s.npy'%activity) else [] #get data if exist
    np.save('facebook_data/fb_%s.npy'%activity,np.append(new_npy,all_events)) #save the new
    return all_events

        
def fb_main(activity, center, search_radius, token=token):
    o = fb_pagefinder(activity, center, search_radius, token=token)
    fb_data = pagination(o, activity)
    

    
def get_df(activity):
    message= "--(fb_%s.npy) doesn't exist--"%activity
    data= 'facebook_data/fb_%s.npy'%activity
    
    fb_data = np.load(data) if os.path.isfile(data) else print(message)
    
    if len(fb_data) != 0:
        fb_info =fb_data[0]['data']
        
        grp_id,name,location,latitude,longitude=[],[],[],[],[]
        
        for i in fb_info:
            grp_id.append(i['id'])
            name.append(i['name'])
            location.append(i['location'])
            
        df = pd.DataFrame([grp_id,name,location]).T
        df.columns=['grp_id','name','location']
        return df
    
    else:
        print('--no data---')
        pass

In [11]:
cities = np.load('main_cities.npy')

In [ ]:
counter = 0
for city in cities:
    counter += 1
    time.sleep(18)
    try:
        fb_main('Weightwatchers', city,16093.4)
    except GraphAPIError as e:
        err1 = 'Please retry your request later'
        err2 = 'request limit reached'
        err3 = 'exceeded the rate limit'

        if err1 in str(e):
            print(e)
            time.sleep(600)
            continue

        elif err2 in str(e):
            print(e)
            time.sleep(600)
            continue

        elif err3 in str(e):
            print(e)
            time.sleep(1800)
            continue
        else:
            raise e

    print('--%s--counter--%d--'%(city,counter))

--[ 51.89107295  -0.42303041]--counter--1--
--[ 51.456659   -0.9696512]--counter--2--
(#613) Calls to this api have exceeded the rate limit.
--[ 52.5738875  -0.2416701]--counter--4--
--[ 53.4018582   -2.56802236]--counter--5--
--[ 50.338466   -4.7882104]--counter--6--
--[ 54.53525645  -1.55294298]--counter--7--
--[ 54.8948478  -2.9362311]--counter--8--
--[ 52.9212617  -1.4761491]--counter--9--
--[ 50.3712659  -4.1425658]--counter--10--
--[ 50.74467035  -1.8579809 ]--counter--11--
--[ 53.7435722  -0.3394758]--counter--12--
--[ 50.84522125  -0.14975851]--counter--13--
--[ 51.6020246  -0.9153487]--counter--14--
--[ 51.8653705  -2.2458192]--counter--15--
(#613) Calls to this api have exceeded the rate limit.
--[ 53.4791301  -2.2441009]--counter--17--
--[ 50.9025349  -1.404189 ]--counter--18--
--[ 52.0553813  -2.7151735]--counter--19--
--[ 51.6553875  -0.3957425]--counter--20--
--[ 50.72414995  -1.17274588]--counter--21--
--[ 51.3876562   0.5457712]--counter--22--
--[ 53.8182212  -3.0564845